In [193]:
# dependencies
import requests
import json
import pandas as pd
import time
import os

# import spotipy
# !pip install spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [194]:
from config import spotify_client_ID as sp_client, spotify_client_secret as sp_secret

In [232]:
# fill-in mini dataset: current top 10 charting songs
# https://www.americantop40.com/charts/top-40-238/october-8-2022/

top10 = pd.read_csv("../00_data/top10.csv")
top10

,week,position,song,artist,features_artist,track_URI
0,8-Oct-22,1,I Like You (A Happier Song),Post Malone,Doja Cat,spotify:track:7JyFwJC3uRpiNfq7oKafgn
1,8-Oct-22,2,Sunroof,Nicky Youre,Dazy,spotify:track:4h4QlmocP3IuwYEj2j14p8
2,8-Oct-22,3,Late Night Talking,Harry Styles,NaN,spotify:track:1qEmFfgcLObUfQm0j1W2CK
3,8-Oct-22,4,About Damn Time,Lizzo,NaN,spotify:track:1PckUlxKqWQs3RlWXVBLw3
4,8-Oct-22,5,As It Was,Harry Styles,NaN,spotify:track:4Dvkj6JhhA12EX05fT7y2e
5,8-Oct-22,6,Vegas,Doja Cat,NaN,spotify:track:0hquQWY3xvYqN4qtiquniF
6,8-Oct-22,7,I Ain't Worried,OneRepublic,NaN,spotify:track:4h9wh7iOZ0GGn8QVp4RAOB
7,8-Oct-22,8,Bad Habit,Steve Lacy,NaN,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK
8,8-Oct-22,9,Glimpse of Us,Joji,NaN,spotify:track:6xGruZOHLs39ZbVccQTuPZ
9,8-Oct-22,10,So Good,Halsey,NaN,spotify:track:6kxaaIeowajN7w21PfMLbu


### Get track audio features from Spotify

In [196]:
# set up client credentials 
# https://spotipy.readthedocs.io/en/master/?highlight=spotifyclientcredentials#client-credentials-flow
# API info: https://developer.spotify.com/documentation/web-api/reference/#/

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(sp_client, sp_secret))

In [5]:
# API info: https://developer.spotify.com/documentation/web-api/
## https://developer.spotify.com/documentation/web-api/reference/#/

sp_root_url = 'https://api.spotify.com/v1/'

In [314]:
# function to get audio features from Spotify
# input a dataframe of top 40 data

def get_audio_features(top40_df):
    
    #
    # get all artist URIs to search for their associated data
    #
    
    # list of all track URIs
    URIs = top40_df['track_URI']
    
    # empty list to hold all artist URIs
    artist_URIs = []
    
    for i in range(len(URIs)):
        
        # get each artist's URI
        artist_URIs.append(sp.track(URIs[i])['album']['artists'][0]['uri'])
    
    # save artist URIs to the dataframe
    top40_df['artist_URI'] = artist_URIs
    
    #
    # get artist genre data
    #
    
    # empty list and dataframe of genres
    top40_genres = []
    
    # empty dictionary to hold all genres associated with each artist
    artist_genre_dict = {}
    
    # get genres associated with all artists    
    for artist in artist_URIs:
        
        # get the artist's genre(s)
        artist_genres = sp.artist(artist)['genres']
        
        # format genre strings
        artist_genres = [x.replace(' ', '_').lower() for x in artist_genres]
        
        artist_genre_dict[artist] = artist_genres
        
        # save new genres to the list of all top40 genres
        for genre in artist_genres:
            if genre not in top40_genres:
                top40_genres.append(genre)
    
                
    # fill the new genre dataframe
    top40_genres_dict = {}
    for genre in top40_genres:
        top40_genres_dict[genre] = [0] * len(top40_df)
    
    # create dataframe with genre data, sort df columns
    genre_df = pd.DataFrame(top40_genres_dict)
    genre_df.columns = sorted(genre_df.columns.tolist())
    genre_df['artist_URI'] = artist_URIs
    genre_df.index = artist_URIs
    
    # assign 1's to each artist's genre                
    for column in genre_df:
        
        for artist in genre_df.index:
            
            for genre in artist_genre_dict[artist]:
                
                if genre == column:
                
                    genre_df.loc[artist, column] = 1
    
    # concatenate all data into one dataframe
    full_df = top40_df.merge(genre_df, how="inner", on="artist_URI")
    full_df.drop_duplicates(inplace=True)
    full_df.sort_values(by="position", inplace=True)
    
    #
    # get audio features of the current top 10 charting songs
    #
    
    # empty lists to hold desired audio feature scores
    acousticness = []
    danceability = []
    energy = []
    instrumentalness = []
    key = []
    liveness = []
    loudness = []
    mode = []
    speechiness = []
    tempo = []
    time_signature = []
    valence = []
    
    artist_features = []
    
    # make the API call for track audio features
    # https://developer.spotify.com/documentation/web-api/reference/#/operations/get-several-audio-features
    for artist in range(len(top40_df)):
        artist_features.append(sp.audio_features(top40_df['track_URI'][artist]))

    # save each feature score to the appropriate list
    for artist in artist_features:
    
        acousticness.append(artist[0]['acousticness'])
        danceability.append(artist[0]['danceability'])
        energy.append(artist[0]['energy'])
        instrumentalness.append(artist[0]['instrumentalness'])
        key.append(artist[0]['key'])
        liveness.append(artist[0]['liveness'])
        loudness.append(artist[0]['loudness'])
        mode.append(artist[0]['mode'])
        speechiness.append(artist[0]['speechiness'])
        tempo.append(artist[0]['tempo'])
        time_signature.append(artist[0]['time_signature'])
        valence.append(artist[0]['valence'])
    
    # save feature data to the dataframe
    full_df['acousticness'] = acousticness
    full_df['danceability'] = danceability
    full_df['energy'] = energy
    full_df['instrumentalness'] = instrumentalness
    full_df['key'] = key
    full_df['liveness'] = liveness
    full_df['loudness'] = loudness
    full_df['mode'] = mode
    full_df['speechiness'] = speechiness
    full_df['tempo'] = tempo
    full_df['time_signature'] = time_signature
    full_df['valence'] = valence


    return full_df

In [315]:
top10_features = get_audio_features(top10)

,week,position,song,artist,features_artist,track_URI,artist_URI,acousticness,danceability,energy,...,etherpop,indie_poptimism,melodic_rap,minnesota_hip_hop,piano_rock,pop,pop_rock,rap,trap_queen,viral_pop
0,8-Oct-22,1,I Like You (A Happier Song),Post Malone,Doja Cat,spotify:track:7JyFwJC3uRpiNfq7oKafgn,spotify:artist:246dkjvS1zLTtiykXe5h60,0.1770,0.301,0.303,...,0,0,1,0,0,0,0,1,0,0
1,8-Oct-22,2,Sunroof,Nicky Youre,Dazy,spotify:track:4h4QlmocP3IuwYEj2j14p8,spotify:artist:7qmpXeNz2ojlMl2EEfkeLs,0.3520,0.768,0.714,...,0,0,0,0,0,0,0,0,0,0
2,8-Oct-22,3,Late Night Talking,Harry Styles,NaN,spotify:track:1qEmFfgcLObUfQm0j1W2CK,spotify:artist:6KImCVD70vtIoJWnq6nGn3,0.2980,0.714,0.728,...,0,0,0,0,0,1,0,0,0,0
6,8-Oct-22,4,About Damn Time,Lizzo,NaN,spotify:track:1PckUlxKqWQs3RlWXVBLw3,spotify:artist:56oDRnqbIiwx4mymNEv7dS,0.0995,0.836,0.743,...,0,0,0,1,0,1,0,0,1,0
4,8-Oct-22,5,As It Was,Harry Styles,NaN,spotify:track:4Dvkj6JhhA12EX05fT7y2e,spotify:artist:6KImCVD70vtIoJWnq6nGn3,0.3420,0.520,0.731,...,0,0,0,0,0,1,0,0,0,0
7,8-Oct-22,6,Vegas,Doja Cat,NaN,spotify:track:0hquQWY3xvYqN4qtiquniF,spotify:artist:5cj0lLjcoR7YOSnhnX0Po5,0.0777,0.801,0.601,...,0,0,0,0,0,1,0,0,0,0
8,8-Oct-22,7,I Ain't Worried,OneRepublic,NaN,spotify:track:4h9wh7iOZ0GGn8QVp4RAOB,spotify:artist:5Pwc4xIPtQLFEnJriah9YJ,0.0826,0.704,0.797,...,0,0,0,0,1,1,1,0,0,0
9,8-Oct-22,8,Bad Habit,Steve Lacy,NaN,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,spotify:artist:57vWImR43h4CaDao012Ofp,0.6260,0.686,0.507,...,0,0,0,0,0,1,0,0,0,0
10,8-Oct-22,9,Glimpse of Us,Joji,NaN,spotify:track:6xGruZOHLs39ZbVccQTuPZ,spotify:artist:3MZsBdqDrRTJihTHQrO6Dq,0.8910,0.440,0.317,...,0,0,0,0,0,0,0,0,0,1
11,8-Oct-22,10,So Good,Halsey,NaN,spotify:track:6kxaaIeowajN7w21PfMLbu,spotify:artist:26VFTg2z8YR0cCuwLzESi2,0.0385,0.576,0.627,...,1,1,0,0,0,1,0,0,0,0


In [317]:
print(top10_features.columns)
top10_features

Index(['week', 'position', 'song', 'artist', 'features_artist', 'track_URI',
       'artist_URI', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence', 'uri',
       'afrofuturism', 'dance_pop', 'dfw_rap', 'electropop', 'escape_room',
       'etherpop', 'indie_poptimism', 'melodic_rap', 'minnesota_hip_hop',
       'piano_rock', 'pop', 'pop_rock', 'rap', 'trap_queen', 'viral_pop'],
      dtype='object')


,week,position,song,artist,features_artist,track_URI,artist_URI,acousticness,danceability,energy,...,etherpop,indie_poptimism,melodic_rap,minnesota_hip_hop,piano_rock,pop,pop_rock,rap,trap_queen,viral_pop
0,8-Oct-22,1,I Like You (A Happier Song),Post Malone,Doja Cat,spotify:track:7JyFwJC3uRpiNfq7oKafgn,spotify:artist:246dkjvS1zLTtiykXe5h60,0.1770,0.301,0.303,...,0,0,1,0,0,0,0,1,0,0
1,8-Oct-22,2,Sunroof,Nicky Youre,Dazy,spotify:track:4h4QlmocP3IuwYEj2j14p8,spotify:artist:7qmpXeNz2ojlMl2EEfkeLs,0.3520,0.768,0.714,...,0,0,0,0,0,0,0,0,0,0
2,8-Oct-22,3,Late Night Talking,Harry Styles,NaN,spotify:track:1qEmFfgcLObUfQm0j1W2CK,spotify:artist:6KImCVD70vtIoJWnq6nGn3,0.2980,0.714,0.728,...,0,0,0,0,0,1,0,0,0,0
6,8-Oct-22,4,About Damn Time,Lizzo,NaN,spotify:track:1PckUlxKqWQs3RlWXVBLw3,spotify:artist:56oDRnqbIiwx4mymNEv7dS,0.0995,0.836,0.743,...,0,0,0,1,0,1,0,0,1,0
4,8-Oct-22,5,As It Was,Harry Styles,NaN,spotify:track:4Dvkj6JhhA12EX05fT7y2e,spotify:artist:6KImCVD70vtIoJWnq6nGn3,0.3420,0.520,0.731,...,0,0,0,0,0,1,0,0,0,0
7,8-Oct-22,6,Vegas,Doja Cat,NaN,spotify:track:0hquQWY3xvYqN4qtiquniF,spotify:artist:5cj0lLjcoR7YOSnhnX0Po5,0.0777,0.801,0.601,...,0,0,0,0,0,1,0,0,0,0
8,8-Oct-22,7,I Ain't Worried,OneRepublic,NaN,spotify:track:4h9wh7iOZ0GGn8QVp4RAOB,spotify:artist:5Pwc4xIPtQLFEnJriah9YJ,0.0826,0.704,0.797,...,0,0,0,0,1,1,1,0,0,0
9,8-Oct-22,8,Bad Habit,Steve Lacy,NaN,spotify:track:5CM4UuQ9Gnd6K2YyKGPMoK,spotify:artist:57vWImR43h4CaDao012Ofp,0.6260,0.686,0.507,...,0,0,0,0,0,1,0,0,0,0
10,8-Oct-22,9,Glimpse of Us,Joji,NaN,spotify:track:6xGruZOHLs39ZbVccQTuPZ,spotify:artist:3MZsBdqDrRTJihTHQrO6Dq,0.8910,0.440,0.317,...,0,0,0,0,0,0,0,0,0,1
11,8-Oct-22,10,So Good,Halsey,NaN,spotify:track:6kxaaIeowajN7w21PfMLbu,spotify:artist:26VFTg2z8YR0cCuwLzESi2,0.0385,0.576,0.627,...,1,1,0,0,0,1,0,0,0,0
